# **MODELO 2 - MODELO DE APRENDIZAJE AUTOMÁTICO BASADO EN REDES NEURONALES CONVOLUCIONALES DE CLASIFICACIÓN MULTICLASE, PARA LA DETECCIÓN AUTOMÁTICA DE EVENTOS**

In [ ]:
# Imports para REDES NEURONALES
import itertools
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from matplotlib import image
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from PIL import Image, ImageDraw
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# IMP: Asumiremos que tenemos el mismo número de imágenes, N, y que tenemos solo 4 tipos de eventos
# Además, el número de trazas de los waterfalls de las 3 opciones, depende de los intervalos que elijamos para hacer el tratamiento de datos correspondiente a cada uno

N=250

In [ ]:
# Manual creation of the classes vectors
numeroImagenes = N
c1 = np.full((1,numeroImagenes), 0)
c2 = np.full((1,numeroImagenes), 1)
c3 = np.full((1,numeroImagenes), 2)
c4 = np.full((1,numeroImagenes), 3)

# IMP: Asumiremos que tenemos el mismo número de imágenes, y que tenemos solo 4 tipos de eventos
# Además, el número de trazas de los waterfalls de las 3 opciones, depende de los intervalos que elijamos para hacer el tratamiento de datos correspondiente a cada uno
classes_V = np.concatenate((c1, c2, c3, c4), axis=1)  # WARNING: transpose, para que funcione asignación de x,y train,test
classes_V = np.reshape(classes_V, (4*numeroImagenes,))
classes_A = classes_V
classes_F = classes_V

#print(classes_A)
print(np.shape(classes_A))

## **1 - RED CONVOLUCIONAL (VARIANZA)**

In [ ]:
# Load in the data
classes = classes_V  
numeroImagenes = N

for indx, i in enumerate(classes):
    if indx == 0:
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/SET 2/Varianza/Ruido/V%d_SoloRuido.npy" %(indx+1)
        images = (np.array([np.loadtxt(filename)])).astype('uint32')
    if indx>0 and indx<=(numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/SET 2/Varianza/Ruido/V%d_SoloRuido.npy" %(indx+1)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=numeroImagenes and indx<=(2*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/SET 2/Varianza/Ruido + Evento Fijo/V%d_RuidoYEventoFijo.npy" %(indx+1-numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=2*numeroImagenes and indx<=(3*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/SET 2/Varianza/Ruido + Andar en paralelo/V%d_RuidoYAndarParalelo.npy" %(indx+1-2*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=3*numeroImagenes and indx<=(4*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/SET 2/Varianza/Ruido + Correr en paralelo/V%d_RuidoYCorrerParalelo.npy" %(indx+1-3*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)

In [ ]:
# Split into data and test

x_train, x_test, y_train, y_test = train_test_split(images, classes, test_size = 0.33, random_state = 46)
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# the data is only 2D!
# convolution expects height x width x color
x_train = np.expand_dims(x_train,-1)  # Añade dimensión para incluir píxel
x_test = np.expand_dims(x_test,-1)  # Añade dimensión para incluir píxel
# x_train shape
print('x_train.shape: ', x_train.shape)  # Por defecto, elegimos 75% de imágenes disponibles para entrenar, y 25% para testear
# number of classes
K = len(set(y_train))
print('number of classes: ', K)

In [ ]:
# Build the model using the TensorFlow 2.0 functional API
i = Input(shape=x_train[0].shape)  # Dimensión de las imágenes
x = Conv2D(32, (9, 9), strides=2, activation='relu')(i)  # 
x = Dropout(0.2)(x)
x = Conv2D(64, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)
model = Model(i,x)

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=12)

In [ ]:
# Plot loss per iteration
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()
plt.savefig('lossv.pdf')

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(cm,classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm,interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max()/2.
  for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,format(cm[i,j], fmt),
             horizontalalignment = 'center',
             color = 'white' if cm[i,j] > thresh else 'black')
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(1)))

## **2 - RED CONVOLUCIONAL (AJUSTE FUNCIONAL)**

In [ ]:
# Load in the data
classes = classes_A  
numeroImagenes = N

for indx, i in enumerate(classes):
    if indx == 0:
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido/A%d_SoloRuido.npy" %(indx+1)
        images = (np.array([np.loadtxt(filename)])).astype('uint32')
    if indx>0 and indx<=(numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido/A%d_SoloRuido.npy" %(indx+1)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=numeroImagenes and indx<=(2*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Evento Fijo/A%d_RuidoYEventoFijo.npy" %(indx+1-numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=2*numeroImagenes and indx<=(3*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Andar en paralelo/A%d_RuidoYAndarParalelo.npy" %(indx+1-2*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=3*numeroImagenes and indx<=(4*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Ajuste/Ruido + Correr en paralelo/A%d_RuidoYCorrerParalelo.npy" %(indx+1-3*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)


In [ ]:
# Split into data and test

x_train, x_test, y_train, y_test = train_test_split(images, classes, test_size = 0.33, random_state = 46)
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# the data is only 2D!
# convolution expects height x width x color
x_train = np.expand_dims(x_train,-1)  # Añade dimensión para incluir píxel
x_test = np.expand_dims(x_test,-1)  # Añade dimensión para incluir píxel
# x_train shape
print('x_train.shape: ', x_train.shape)  # Por defecto, elegimos 75% de imágenes disponibles para entrenar, y 25% para testear
# number of classes
K = len(set(y_train))
print('number of classes: ', K)

In [ ]:
# Build the model using the TensorFlow 2.0 functional API
i = Input(shape=x_train[0].shape)  # Dimensión de las imágenes
x = Conv2D(32, (9, 9), strides=2, activation='relu')(i)  # 
x = Dropout(0.2)(x)
x = Conv2D(64, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)
model = Model(i,x)

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=12)

In [ ]:
# Plot loss per iteration
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(cm,classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm,interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max()/2.
  for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,format(cm[i,j], fmt),
             horizontalalignment = 'center',
             color = 'white' if cm[i,j] > thresh else 'black')
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(1)))

## **3 - RED CONVOLUCIONAL (FOURIER)**

In [ ]:
# Load in the data
classes = classes_F  
numeroImagenes = N

for indx, i in enumerate(classes):
    if indx == 0:
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido/F%d_SoloRuido.npy" %(indx+1)
        images = (np.array([np.loadtxt(filename)])).astype('uint32')
    if indx>0 and indx<=(numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido/F%d_SoloRuido.npy" %(indx+1)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=numeroImagenes and indx<=(2*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Evento Fijo/F%d_RuidoYEventoFijo.npy" %(indx+1-numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=2*numeroImagenes and indx<=(3*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Andar en paralelo/F%d_RuidoYAndarParalelo.npy" %(indx+1-2*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)
    if indx>=3*numeroImagenes and indx<=(4*numeroImagenes-1):
        filename = "/content/drive/MyDrive/TFG_COTDR_Imágenes/Fourier/Ruido + Correr en paralelo/F%d_RuidoYCorrerParalelo.npy" %(indx+1-3*numeroImagenes)
        images = np.concatenate((images,(np.array([np.loadtxt(filename)])).astype('uint32')),axis=0)

In [ ]:
# Split into data and test

x_train, x_test, y_train, y_test = train_test_split(images, classes, test_size = 0.33, random_state = 46)
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# the data is only 2D!
# convolution expects height x width x color
x_train = np.expand_dims(x_train,-1)  # Añade dimensión para incluir píxel
x_test = np.expand_dims(x_test,-1)  # Añade dimensión para incluir píxel
# x_train shape
print('x_train.shape: ', x_train.shape)  # Por defecto, elegimos 75% de imágenes disponibles para entrenar, y 25% para testear
# number of classes
K = len(set(y_train))
print('number of classes: ', K)

In [ ]:
# Build the model using the TensorFlow 2.0 functional API
i = Input(shape=x_train[0].shape)  # Dimensión de las imágenes
x = Conv2D(32, (9, 9), strides=2, activation='relu')(i)  # 
x = Dropout(0.2)(x)
x = Conv2D(64, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, (9, 9), strides=2, activation='relu')(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)
model = Model(i,x)

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=12)

In [ ]:
# Plot loss per iteration
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(cm,classes,
                          normalize=False,
                          title='Matriz de confusión',
                          cmap=plt.cm.Blues):
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm,interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max()/2.
  for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,format(cm[i,j], fmt),
             horizontalalignment = 'center',
             color = 'white' if cm[i,j] > thresh else 'black')
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(1)))